In [41]:
# Célula 1 – Importações e leitura dos dados

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

# Leitura do novo CSV com os dados simulados
df = pd.read_csv("dados_meteorologicos.csv")  # substitua pelo nome real se for diferente
df.head()


,temperatura_c,umidade_percent,pressao_hpa,distancia_cm,risco_tempestade
0,45.64,82.87,995.50,400.09,0
1,26.25,86.38,998.28,399.99,0
2,45.12,84.26,987.23,400.09,0
3,48.99,75.62,995.62,399.90,0
4,79.66,70.06,981.19,400.04,1


In [42]:
# Célula 2 – Criação da coluna risco_tempestade com base em limiares físicos

# Garante que os dados estejam no tipo numérico
df["temperatura_c"] = pd.to_numeric(df["temperatura_c"], errors="coerce")
df["umidade_percent"] = pd.to_numeric(df["umidade_percent"], errors="coerce")
df["pressao_hpa"] = pd.to_numeric(df["pressao_hpa"], errors="coerce")

# Remove linhas com dados faltando (se houver)
df = df.dropna()

# Define a condição de risco
df["risco_tempestade"] = df.apply(
    lambda row: 1 if (row["pressao_hpa"] < 1005 and row["umidade_percent"] > 70 and row["temperatura_c"] > 27) else 0,
    axis=1
)

# Visualiza a distribuição
df["risco_tempestade"].value_counts()


risco_tempestade
0    2500
1    1500
Name: count, dtype: int64

In [43]:
# Célula 3 – Criação das variáveis X e y

# Define as variáveis independentes (entradas do modelo)
X = df[["temperatura_c", "umidade_percent", "pressao_hpa", "distancia_cm"]]

# Define a variável dependente (alvo que queremos prever)
y = df["risco_tempestade"]

# Exibe uma amostra para conferência
X.head(), y.head()

(   temperatura_c  umidade_percent  pressao_hpa  distancia_cm
 0          45.64            82.87       995.50        400.09
 1          26.25            86.38       998.28        399.99
 2          45.12            84.26       987.23        400.09
 3          48.99            75.62       995.62        399.90
 4          79.66            70.06       981.19        400.04,
 0    1
 1    0
 2    1
 3    1
 4    1
 Name: risco_tempestade, dtype: int64)

In [44]:
# Célula 4 – Treinamento do modelo

# Divide os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cria e treina o modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [45]:
# Célula 5 – Avaliação da acurácia

# Faz previsões nos dados de teste
y_pred = modelo.predict(X_test)

# Calcula a acurácia
acuracia = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {acuracia:.2f}")

Acurácia do modelo: 1.00


In [46]:
# Célula 6 – Exportação com pickle

with open("modelo_risco_tempestade.pkl", "wb") as f:
    pickle.dump(modelo, f)

print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!


In [47]:
# Célula 7 – Previsão simulada com modelo salvo

import numpy as np

# Carrega o modelo salvo
with open("modelo_risco_tempestade.pkl", "rb") as f:
    modelo_carregado = pickle.load(f)

# Exemplo de novo dado do ESP32 (temp, umidade, pressao, distancia)
novo_dado = np.array([[28.5, 72.3, 1002.5, 400.0]])

# Faz a previsão
previsao = modelo_carregado.predict(novo_dado)

# Resultado
if previsao[0] == 1:
    print("⚠️ RISCO DE TEMPESTADE PREVISTO!")
else:
    print("✅ Sem risco de tempestade previsto.")


⚠️ RISCO DE TEMPESTADE PREVISTO!


c:\Users\tiago\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [48]:
# Célula 8 – Simulação de integração com ESP32 (dados contínuos)

import time

# Carrega novamente o modelo treinado (se necessário)
with open("modelo_risco_tempestade.pkl", "rb") as f:
    modelo = pickle.load(f)

# Lê o CSV com os dados simulados do ESP32
df = pd.read_csv("dados_meteorologicos.csv")

# Simula recebimento contínuo de dados linha a linha
for index, row in df.iterrows():
    entrada = [[row["temperatura_c"], row["umidade_percent"], row["pressao_hpa"], row["distancia_cm"]]]
    previsao = modelo.predict(entrada)

    print(f"🌡️ Temp: {row['temperatura_c']} °C | 💧 Umidade: {row['umidade_percent']} % | 🌪️ Pressão: {row['pressao_hpa']} hPa")

    if previsao[0] == 1:
        print("⚠️ RISCO DE TEMPESTADE!")
    else:
        print("✅ Sem risco de tempestade no momento.")

    print("-" * 40)
    time.sleep(1)  # Simula tempo real

c:\Users\tiago\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


🌡️ Temp: 45.64 °C | 💧 Umidade: 82.87 % | 🌪️ Pressão: 995.5 hPa
⚠️ RISCO DE TEMPESTADE!
----------------------------------------


c:\Users\tiago\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


🌡️ Temp: 26.25 °C | 💧 Umidade: 86.38 % | 🌪️ Pressão: 998.28 hPa
✅ Sem risco de tempestade no momento.
----------------------------------------


c:\Users\tiago\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


...

c:\Users\tiago\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


🌡️ Temp: 26.88 °C | 💧 Umidade: 80.27 % | 🌪️ Pressão: 983.85 hPa
✅ Sem risco de tempestade no momento.
----------------------------------------


c:\Users\tiago\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


🌡️ Temp: 74.19 °C | 💧 Umidade: 68.85 % | 🌪️ Pressão: 998.31 hPa
✅ Sem risco de tempestade no momento.
----------------------------------------


KeyboardInterrupt: 